## Fake News Detection with Python

This project aims at detecting fake news thorugh machine learning some real data. The dataset used is available at: https://data-flair.training/blogs/advanced-python-project-detecting-fake-news/

In [1]:
# importing libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline

In [2]:
# reading the data file
news = pd.read_csv('news.csv',index_col=0)

In [3]:
news.head()

,title,text,label
8476,You Can Smell Hillary’s Fear,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
10294,Watch The Exact Moment Paul Ryan Committed Pol...,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
3608,Kerry to go to Paris in gesture of sympathy,U.S. Secretary of State John F. Kerry said Mon...,REAL
10142,Bernie supporters on Twitter erupt in anger ag...,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
875,The Battle of New York: Why This Primary Matters,It's primary day in New York and front-runners...,REAL


### Exploratory Data Analysis

In [4]:
news.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 6335 entries, 8476 to 4330
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   title   6335 non-null   object
 1   text    6335 non-null   object
 2   label   6335 non-null   object
dtypes: object(3)
memory usage: 198.0+ KB


There are no missing values

In [5]:
news.groupby('label').count()

,title,text
label,,
FAKE,3164,3164
REAL,3171,3171


Real and fake pieces of news are well balanced in number.  Next, we study text length:

In [6]:
news[news['label']=='REAL']['text'].map(len).describe()

count     3171.000000
mean      5292.160202
std       4348.288284
min         43.000000
25%       2729.500000
50%       4683.000000
75%       6829.500000
max      44039.000000
Name: text, dtype: float64

In [7]:
news[news['label']=='FAKE']['text'].map(len).describe()

count      3164.000000
mean       4121.046460
std        5680.232733
min           1.000000
25%        1283.500000
50%        2558.000000
75%        5027.000000
max      115372.000000
Name: text, dtype: float64

Fake news has higher varaince of text length, i.e., extremely short or long texts are more often fake. This is also reflceted by the respective minimum and maximum values.

### Data pre-processing 

We will use CountVectorizer to convert a text into a vector. It cannot handle two different columns of texts. Therefore, we will merge the 'title' and 'text' columns into one. Dropping the 'title' column altogether may eliminate valuable information.

In [8]:
# merging title and text into one text

news['text'] = news['title']+' '+news['text']
del news['title']

Next, we convert label into 0 and 1, which correspond respectively to real and fake.

In [9]:
# label conversion

news['label'] = news['label'].map({'REAL': 0,'FAKE': 1})

In [10]:
news.head()

,text,label
8476,You Can Smell Hillary’s Fear Daniel Greenfield...,1
10294,Watch The Exact Moment Paul Ryan Committed Pol...,1
3608,Kerry to go to Paris in gesture of sympathy U....,0
10142,Bernie supporters on Twitter erupt in anger ag...,1
875,The Battle of New York: Why This Primary Matte...,0


Now we use natural language toolkit to clean up the texts  

In [11]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
#from nltk.stem.porter import PorterStemmer
import string

def text_preprocess(text):
    """
    Takes in a string of text, then performs the following:
    1. Remove all punctuation
    2. Remove all stopwords
    3. Returns cleaned text in as string
    """
    # removing punctuation
    no_punc = [char for char in text if char not in string.punctuation]

    # rejoining the characters to form the string
    no_punc = ''.join(no_punc)
    
    # removing stopwords
    clean = [word for word in no_punc.split() if word.lower() not in stopwords.words('english')]
    
    # rejoining characters to form string
    clean = ' '.join(clean)
    
    return clean

[nltk_data] Downloading package stopwords to C:\Users\Rakibur
[nltk_data]     Rahman\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


It could be seen a posteriori that stemming the words would lower the performance of our models. We do not go for it. Now we execute text pre-processing:

In [12]:
import time
t1 = time.time()

news['text'] = news['text'].apply(text_preprocess)

t2 = time.time()
print('Time taken: {:.1f} sec'.format(t2-t1))

Time taken: 1620.5 sec


Finally, we do the train-test splitting:

In [13]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test =\
train_test_split(news['text'], news['label'],test_size=0.2, random_state=101)

### Cross-validation with default hyperparameters

In [14]:
# import the various classifiers to be expored

from sklearn.linear_model import LogisticRegression, PassiveAggressiveClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier

clf = {'Logistic Regression': LogisticRegression(),
       'Passive-Aggressive': PassiveAggressiveClassifier(),
       'K Neighbors': KNeighborsClassifier(), 
       'Support Vector': SVC(), 
       'Multinomial NB': MultinomialNB(),
       'Decision Tree': DecisionTreeClassifier(), 
       'Random Forest': RandomForestClassifier(), 
       'XG Boost': XGBClassifier(use_label_encoder=False, eval_metric='auc')}

In [15]:
# define a function for computing cross-validated training accuracy 

from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import accuracy_score

def cross_val_report(clf_,X_,y_,cv_):
    '''This function gives cross-validated training accuracy for given classifier'''
    ti = time.time()
    # pipeline is created to vectorize, Tfidf transform and then classify 
    pipeline = Pipeline([('bow',CountVectorizer()), ('tfidf',TfidfTransformer()), ('clf',clf_)])
    acc_list = cross_val_score(pipeline, X_, y_, cv=cv_, scoring='accuracy')
    mean_ = round(100*acc_list.mean(),1)
    err_ = round(100*np.sqrt(acc_list.std()/(cv_-1)),1)
    accuracy_ = str(mean_)+' $\pm$ '+str(err_)
    
    tf = time.time()
    time_ = round(tf-ti,1)
    
    return accuracy_, time_

Training accuracy is computed for all classfiers 

In [16]:
accuracy = []
time_taken = []

for i in range(len(clf)):
    accuracy_, time_ = \
    cross_val_report(list(clf.values())[i],X_train,y_train,10)
    accuracy.append(accuracy_)
    time_taken.append(time_)
    print(list(clf.keys())[i]+' took {} sec'.format(time_))

Logistic Regression took 28.9 sec
Passive-Aggressive took 23.5 sec
K Neighbors took 24.8 sec
Support Vector took 431.9 sec
Multinomial NB took 22.7 sec
Decision Tree took 67.8 sec
Random Forest took 74.0 sec
XG Boost took 302.8 sec


In [17]:
# dataframe for cross validation

cross_val_data = {'Train Accuracy (%)': accuracy, 'Time (sec)': time_taken}
df1 = pd.DataFrame(index=clf.keys(), data=cross_val_data, columns=cross_val_data.keys())

In [18]:
df1

,Train Accuracy (%),Time (sec)
Logistic Regression,91.5 $\pm$ 3.8,28.9
Passive-Aggressive,93.8 $\pm$ 3.0,23.5
K Neighbors,84.7 $\pm$ 3.6,24.8
Support Vector,93.1 $\pm$ 3.0,431.9
Multinomial NB,83.2 $\pm$ 3.6,22.7
Decision Tree,80.7 $\pm$ 4.1,67.8
Random Forest,90.2 $\pm$ 2.5,74.0
XG Boost,92.6 $\pm$ 3.7,302.8


### Hyperparameter tuning

In [19]:
# Dictionary of tunable hyperparameters of various classifiers

from scipy.stats import expon

hp = {'K Neighbors': dict(n_neighbors = list(range(1,31)),
                          metric = ['euclidean', 'manhattan', 'minkowski'], 
                          weights = ['uniform', 'distance']),
      'Passive-Aggressive': dict(C = 10.**np.arange(-3, 3),
                                 max_iter = list(range(1_000,10_100,100))),
      'Logistic Regression': dict(C = 10.**np.arange(-3, 3),
                                  penalty = ['l1', 'l2', 'elasticnet']),
      'Decision Tree': dict(criterion = ['gini', 'entropy'],
                            min_samples_leaf = list(range(1,10)),
                            max_depth = list(range(2,14,2))),
      'Support Vector': dict(C = expon(scale=100),      # param distribution for SVC 
                             gamma = expon(scale=.1)),
      'Random Forest': dict(n_estimators = [10,50,100,500,1000],
                            max_features = ['auto', 'sqrt', 'log2']),
      'XG Boost': dict(learning_rate = 0.1*np.arange(1,11), # default = 0.3
                       max_depth = list(range(2,31,2)),  # default = 6
                       min_child_weight = 10.**np.arange(-2,3), # default = 1
                       min_split_loss = 0.1**np.arange(0,5), # default = 0
                       colsample_bytree = 0.1*np.arange(1,11)), # default = 1
      'Multinomial NB': dict(alpha=[1.0], fit_prior=[True], class_prior=[None])} # default values

In [20]:
# creating a dictionary for best estimators
best_values = {}

In [21]:
# define function for grid serach/randomized serach of best hyperparameters

from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

def best_estimator(clf_name,X_,y_,cv_,rs_=False,n_iter=None):
    '''This function finds best estimator for a given classifier'''
    ti = time.time()
    if rs_:
        grid = RandomizedSearchCV(clf[clf_name], param_distributions=hp[clf_name], 
                                  n_iter=n_iter, scoring='accuracy', n_jobs=-1, cv=cv_)
    else:
        grid = GridSearchCV(clf[clf_name], param_grid=hp[clf_name], 
                        scoring='accuracy', n_jobs=-1, cv=cv_)
    pipeline = Pipeline([('bow',CountVectorizer()),
                         ('tfidf',TfidfTransformer()), 
                         ('grid',grid)])
    pipeline.fit(X_,y_)
    
    best_score = round(100*grid.best_score_,1)
    best_pipeline = Pipeline([('bow',CountVectorizer()),
                              ('tfidf',TfidfTransformer()),
                              ('best_est',grid.best_estimator_)])
    
    tf = time.time()
    time_ = round(tf-ti,1)
    
    best_values[clf_name] = [best_score, best_pipeline, time_]
    
    print(clf_name+' took '+str(time_)+' sec; best train accuracy: '+str(best_score)+'%')

In [22]:
# hyperparameter tuning for Logistic Regression

best_estimator('Logistic Regression',X_train,y_train,cv_=10)

Logistic Regression took 20.2 sec; best train accuracy: 93.7%


In [23]:
# hyperparameter tuning for Passive-Aggressive

best_estimator('Passive-Aggressive',X_train,y_train,cv_=10)

Passive-Aggressive took 617.4 sec; best train accuracy: 94.0%


In [24]:
# hyperparameter tuning for K Neighbors

best_estimator('K Neighbors',X_train,y_train,cv_=10)

K Neighbors took 1285.2 sec; best train accuracy: 86.1%


In [25]:
# hyperparameter tuning for Support vector

best_estimator('Support Vector',X_train,y_train,cv_=10,rs_=True,n_iter=10)

Support Vector took 1061.6 sec; best train accuracy: 93.9%


In [26]:
# Multinomial NB (no tuning, just default hyperparameters)

best_estimator('Multinomial NB',X_train,y_train,cv_=10)

Multinomial NB took 2.6 sec; best train accuracy: 83.4%


In [27]:
# hyperparameter tuning for Decision Tree

best_estimator('Decision Tree',X_train,y_train,cv_=10)

Decision Tree took 628.5 sec; best train accuracy: 82.2%


In [28]:
# hyperparameter tuning for Random Forest

best_estimator('Random Forest',X_train,y_train,cv_=10)

Random Forest took 1316.9 sec; best train accuracy: 91.5%


In [29]:
# hyperparameter tuning for XG Boost

best_estimator('XG Boost',X_train,y_train,cv_=10,rs_=True,n_iter=15)

XG Boost took 2459.0 sec; best train accuracy: 92.2%


In [30]:
# dataframe for best training scores

def best_list(index):
    return [list(best_values.values())[i][index] for i in range(len(best_values))]
        
best_new = {'Best Score (%)': best_list(0), 'Search Time (sec)': best_list(2)}
df2 = pd.DataFrame(index=best_values.keys(), data=best_new, columns=best_new.keys())

# inclusion of default performance
df_best = pd.concat([df2,df1['Train Accuracy (%)'].apply(lambda i: float(i[:4]))], axis=1)
df_best.rename(columns={'Train Accuracy (%)':'Default Accuracy (%)'}, inplace=True)

# null accuracy for training data
null_train = round(100*max(y_train.mean(),1-y_train.mean()),1)

# discard bad classifiers, if any, which give accuracy close to the null value
df_best = df_best[df_best['Best Score (%)']>(null_train+5.0)]

In [31]:
df_best

,Best Score (%),Search Time (sec),Default Accuracy (%)
Logistic Regression,93.7,20.2,91.5
Passive-Aggressive,94.0,617.4,93.8
K Neighbors,86.1,1285.2,84.7
Support Vector,93.9,1061.6,93.1
Multinomial NB,83.4,2.6,83.2
Decision Tree,82.2,628.5,80.7
Random Forest,91.5,1316.9,90.2
XG Boost,92.2,2459.0,92.6


### Performance on test data

In [32]:
# define function for evaluating various metrics

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score 

def test_result(clf_name,X_,y_,X_new,y_new):
    '''This function evaluates various performance metrics on test data'''

    best_pipeline = best_values[clf_name][1]
    best_pipeline.fit(X_,y_)
    y_pred = best_pipeline.predict(X_new)
    
    accuracy = round(accuracy_score(y_new,y_pred),3)
    precision = round(precision_score(y_new,y_pred),3)
    recall = round(recall_score(y_new,y_pred),3)
    f1 = round(f1_score(y_new,y_pred),3)
    
    return accuracy, precision,recall, f1

In [33]:
# create a dataframe of performance metrics

metrics = {'Accuracy': [], 'Precision': [], 'Recall': [], 'f1 Score': []}
for estimator in df_best.index:
    accuracy,precision,recall,f1 = test_result(estimator,X_train,y_train,X_test,y_test)
    metrics['Accuracy'].append(accuracy)
    metrics['Precision'].append(precision)
    metrics['Recall'].append(recall)
    metrics['f1 Score'].append(f1)

df_result = pd.DataFrame(index=df_best.index, data=metrics, columns=metrics.keys())

In [34]:
df_result

,Accuracy,Precision,Recall,f1 Score
Logistic Regression,0.940,0.914,0.967,0.940
Passive-Aggressive,0.944,0.926,0.961,0.943
K Neighbors,0.867,0.926,0.790,0.852
Support Vector,0.934,0.910,0.959,0.934
Multinomial NB,0.846,0.975,0.700,0.815
Decision Tree,0.796,0.764,0.837,0.799
Random Forest,0.916,0.911,0.917,0.914
XG Boost,0.919,0.911,0.922,0.917


Passive Aggressive Classifier is the best performer.
Liner Regression is the second best.

In [35]:
# null accuracy for test data

null_test = round(100*max(y_test.mean(),1-y_test.mean()),1)
print('Null accuracy for test data: {}%'.format(null_test))

Null accuracy for test data: 51.5%


### Voting classifier of 3 estimators

In [36]:
from sklearn.ensemble import VotingClassifier

def voting_(voter_list,vote_type):
    '''This function creates a voting classifier of 3 estimators,
    which is subsequently trained and tested'''
    
    # creating voting classifier
    estimator_list = []
    for clf_name in voter_list:
        estimator_list.append((clf_name,best_values[clf_name][1]))
    voting_clf = VotingClassifier(estimators=estimator_list,voting=vote_type)
    
    # training and testing
    voting_clf.fit(X_train,y_train)
    y_pred = voting_clf.predict(X_test)
    
    # evaluation of metrics
    accuracy = round(accuracy_score(y_test,y_pred),3)
    precision = round(precision_score(y_test,y_pred),3)
    recall = round(recall_score(y_test,y_pred),3)
    f1 = round(f1_score(y_test,y_pred),3)
    
    return accuracy, precision, recall, f1

In the voter list, let us choose the bottom 3 performers (in terms of accuracy with training data

In [37]:
# create voter list
voter_list = []
cutoff = sorted(df_best['Best Score (%)'])[2]
for estimator in df_best.index:
    if df_best['Best Score (%)'][estimator] <= cutoff:
        voter_list.append(estimator)
print('voter list: ', voter_list)

voter list:  ['K Neighbors', 'Multinomial NB', 'Decision Tree']


In [38]:
# create a dataframe of performance metrics

voting_metrics = {'Accuracy': [], 'Precision': [], 'Recall': [], 'f1 Score': []}
for vote_type in ['hard','soft']:
        accuracy, precision, recall, f1 = voting_(voter_list,vote_type)
        voting_metrics['Accuracy'].append(accuracy)
        voting_metrics['Precision'].append(precision)
        voting_metrics['Recall'].append(recall)
        voting_metrics['f1 Score'].append(f1)

df_voting = pd.DataFrame(index=['Hard Voting','Soft Voting'],
                         data=voting_metrics, columns=voting_metrics.keys())

In [39]:
df_voting

,Accuracy,Precision,Recall,f1 Score
Hard Voting,0.892,0.960,0.811,0.879
Soft Voting,0.890,0.931,0.834,0.880


Voting classifier has much more improved predictions than individual voters